# Adjoint-based Optimization for an Annulus Domain with Viscoplastic Rheology

In this exercise, we extend our understanding of adjoint-based optimization to a more complex geophysical scenario. Unlike the previous exercise that focused on a simple square domain, here we explore an annulus domain, mimicking Earth's mantle. We also introduce a more sophisticated viscoplastic rheology model. 

Key differences from the previous exercise:
1. **Geometry**: We are working with an annulus domain.
2. **Rheology**: We introduce a viscoplastic model.

Let's dive in! But before that, let's make sure we have all the dependencies. 


Here, we first import the required libraries. We then get the tape from `pyadjoint` using `get_working_tape()` and clear it using `tape.clear_tape()`. This ensures that the tape is clean before we start, which is particularly important when running multiple instances of adjoint simulations.


In this section, we set up the geometry of our annulus domain. We define $r_{max}=2.2$, $r_{min}=1.2$. This ensures a unit length for the thickness of the domain, while mainating the same ratio between surface and CMB as it is for the Earth. We also obtain the non-dimensional radius of 410 and 660 discontinuities in this email, which will be used to implement the depth dependence of  km depths, which represent viscosity jumps.



In [ ]:
# Set up geometry:
 rmax = 2.22
 rmax_earth = 6370  # Radius of Earth [km]
 rmin_earth = rmax_earth - 2900  # Radius of CMB [km]
 r_410_earth = rmax_earth - 410  # 410 radius [km]
 r_660_earth = rmax_earth - 660  # 660 raidus [km]
 r_410 = rmax - (rmax_earth - r_410_earth) / (rmax_earth - rmin_earth)
 r_660 = rmax - (rmax_earth - r_660_earth) / (rmax_earth - rmin_earth)


We load the mesh from our reference simulation stored in `Checkpoint230.h5`. After that, we enable disk checkpointing for adjoint intermediary fields. This is crucial for managing memory efficiently, especially for large problems.


In [ ]:
with CheckpointFile("Checkpoint230.h5", "r") as f:
        mesh = f.load_mesh("firedrake_default_extruded")

    enable_disk_checkpointing()

We define various function spaces for our problem. Importantly, we use \( Q2 \) elements for temperature but \( Q1 \) elements for our control variable. 


In [ ]:
# Set up function spaces for the Q2Q1 pair
V = VectorFunctionSpace(mesh, "CG", 2)  # Velocity function space (vector)
W = FunctionSpace(mesh, "CG", 1)  # Pressure function space (scalar)
Q = FunctionSpace(mesh, "CG", 2)  # Temperature function space (scalar)
Q1 = FunctionSpace(mesh, "CG", 1)  # Control function space
Z = MixedFunctionSpace([V, W])  # Mixed function space

# Test functions and functions to hold solutions:
z = Function(Z)  # A field over the mixed function space Z
u, p = split(z)  # Symbolic UFL expressions for u and p

X = SpatialCoordinate(mesh)
r = sqrt(X[0] ** 2 + X[1] ** 2)

We set up the Rayleigh number and the Boussinesq Approximation here. We also define the time-stepping parameters and load our control variable $\text{Tic}$ and $\text{Taverage}$ for regularisation.



In [ ]:
Ra = Constant(1e7)  # Rayleigh number
approximation = BoussinesqApproximation(Ra)

# Define time stepping parameters:
max_timesteps = 200
delta_t = Constant(5e-6)  # Constant time step

# Without a restart to continue from, our initial guess is the final state of the forward run
# We need to project the state from Q2 into Q1
Tic = Function(Q1, name="Initial Temperature")
Taverage = Function(Q1, name="Average Temperature")

checkpoint_file = CheckpointFile("Checkpoint_State.h5", "r")
# Initialise the control
Tic.project(
    checkpoint_file.load_function(mesh, "Temperature", idx=max_timesteps - 1)
)
Taverage.project(checkpoint_file.load_function(mesh, "Average Temperature", idx=0))

# Temperature function in Q2, where we solve the equations
T = Function(Q, name="Temperature")

: 

In this section, we build a complex viscoplastic rheology model. We use a step function to represent the viscosity jumps at 410 and 660 km depths. The rheology has two components: a linear part that is temperature-dependent and a plastic part that is rheology-dependent.

\[
\text{Equations for viscosity here}
\]


Finally, we set up the solvers for the Stokes and energy equations. Doing that we also set up nullspaces and near-nullspaces for efficient solving of the system.

In [ ]:
# Nullspaces and near-nullspaces:
Z_nullspace = create_stokes_nullspace(Z, closed=True, rotational=True)
Z_near_nullspace = create_stokes_nullspace(
    Z, closed=False, rotational=True, translations=[0, 1]
)

stokes_bcs = {
    "top": {"un": 0},
    "bottom": {"un": 0},
}
temp_bcs = {
    "top": {"T": 0.0},
    "bottom": {"T": 1.0},
}

energy_solver = EnergySolver(
    T,
    u,
    approximation,
    delta_t,
    ImplicitMidpoint,
    bcs=temp_bcs,
)

stokes_solver = StokesSolver(
    z,
    T,
    approximation,
    mu=mu,
    bcs=stokes_bcs,
    cartesian=False,
    nullspace=Z_nullspace,
    transpose_nullspace=Z_nullspace,
    near_nullspace=Z_near_nullspace,
    solver_parameters=newton_stokes_solver_parameters,
)

The control variable for our optimisation problem is set to be `Tic`, which represents the initial condition for temperature. This is the parameter we aim to optimise in our inverse problem.



In [ ]:
# Control variable for optimisation
control = Control(Tic)

We initialize `u_misfit` to zero. This variable will be used to accumulate the misfit between the observed and simulated surface velocity as we proceed through the time steps.


In [ ]:
u_misfit = 0.0

Next, we need to project the initial condition `Tic` from the $Q1$ space to $Q2$ space. This is necessary because the temperature is solved in $Q2$ space. We also impose boundary conditions while doing this projection.
After the projection, we are ready to run the forward simulation to populate the tape, which will be used for adjoint computations later. For each time step, we solve both the Stokes and energy equations. We then update the variable `u_misfit` to accumulate the surface velocity misfit using the observed values. While running the forward problem, we also load reference velocity fields information that will be used in the objective function:




In [ ]:
T.project(Tic, bcs=energy_solver.strong_bcs)
for timestep in range(0, max_timesteps):
    stokes_solver.solve()
    energy_solver.solve()
    uobs = checkpoint_file.load_function(mesh, name="Velocity", idx=timestep)
    u_misfit += assemble(dot(u - uobs, u - uobs) * ds_t)

We also load various fields
- `Tobs`: The observed final state of temperature.
- `Tic_ref`: The reference initial state of temperature, used to measure the performance of an inverse scheme.
- `Taverage`: The average temperature profile, used for regularization.

We then close the checkpoint file.

In [ ]:
# Load the observed final state
Tobs = checkpoint_file.load_function(mesh, "Temperature", idx=max_timesteps - 1)
Tobs.rename("Observed Temperature")

# Load the reference initial state
# Needed to measure performance of weightings
Tic_ref = checkpoint_file.load_function(mesh, "Temperature", idx=0)
Tic_ref.rename("Reference Initial Temperature")

# Load the average temperature profile
Taverage = checkpoint_file.load_function(mesh, "Average Temperature", idx=0)

checkpoint_file.close()

We define the components of the objective functional, which include terms for damping, smoothing, and the misfits for temperature and surface velocity. These are combined to form the overall objective function that we aim to minimize.

In [ ]:
# Define the component terms of the overall objective functional
damping = assemble((Tic - Taverage) ** 2 * dx)
norm_damping = assemble(Taverage**2 * dx)
smoothing = assemble(dot(grad(Tic - Taverage), grad(Tic - Taverage)) * dx)
norm_smoothing = assemble(dot(grad(Tobs), grad(Tobs)) * dx)
norm_obs = assemble(Tobs**2 * dx)
norm_u_surface = assemble(dot(uobs, uobs) * ds_t)

# Temperature misfit between solution and observation
t_misfit = assemble((T - Tobs) ** 2 * dx)

objective = (
    t_misfit +
    alpha_u * (norm_obs * u_misfit / max_timesteps / norm_u_surface) +
    alpha_d * (norm_obs * damping / norm_damping) +
    alpha_s * (norm_obs * smoothing / norm_smoothing)
)

We pause the annotation to the tape as we are done with the forward run. This ensures that no further operations are added to the tape, which will be used for adjoint calculations.
We define the reduced functional, which represents the objective function with respect to the control variable. This will be used for optimization.
We define a callback function to log the initial and final misfits as the optimization progresses. This is useful for monitoring the optimization process.
We set up a bounded nonlinear optimization problem. The temperature is restricted to be within the range [0, 1]. We then define the minimization problem with these bounds.


In [ ]:
# All done with the forward run, stop annotating anything else to the tape
pause_annotation()
# Defining the object for pyadjoint
reduced_functional = ReducedFunctional(objective, control)
def callback():
    initial_misfit = assemble(
        (Tic.block_variable.checkpoint.restore() - Tic_ref) ** 2 * dx
    )
    final_misfit = assemble(
        (T.block_variable.checkpoint.restore() - Tobs) ** 2 * dx
    )

    log(f"Initial misfit; {initial_misfit}; final misfit: {final_misfit}")
# Perform a bounded nonlinear optimisation where temperature
# is only permitted to lie in the range [0, 1]
T_lb = Function(Tic.function_space(), name="Lower bound temperature")
T_ub = Function(Tic.function_space(), name="Upper bound temperature")
T_lb.assign(0.0)
T_ub.assign(1.0)

minimisation_problem = MinimizationProblem(reduced_functional, bounds=(T_lb, T_ub))


Finally, we set up and run the optimizer. We also add the callback function to log the misfits during the optimization.
If multiple successive optimizations are being performed, it's important to resume annotation to the tape for the next run.


In [ ]:
optimiser = LinMoreOptimiser(
    minimisation_problem,
    minimisation_parameters,
    checkpoint_dir="optimisation_checkpoint",
)
optimiser.add_callback(callback)
optimiser.run()

## Conclusion

In this exercise, we have delved into a more complex geophysical scenario with a focus on an annulus domain and viscoplastic rheology. This adds layers of realism to our inversion problem and gives us a deeper understanding of the complexities involved in geodynamics simulations.

Key Takeaways:
1. The geometry and rheology significantly impact the inversion process.
2. Sophisticated rheology models can be implemented efficiently using Firedrake and pyadjoint.

